In [1]:
# end_year = 인곳만 변경되게 적용

# DB 방식으로 변경: CSV 읽어서 documents 테이블의 사원 정보 업데이트
# activities, referrers JSON 업데이트 + drafter 단일 컬럼 업데이트
# *** 2015년 문서만 업데이트 ***

import pandas as pd
import json
import pymysql

# ═══════════════════════════════════════════════════════════
# 설정
# ═══════════════════════════════════════════════════════════

# DB 설정
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'any_approval'
}

CSV_FILE = '인사정보_부서코드추가.csv'

# ═══════════════════════════════════════════════════════════
# 1. CSV 파일 읽기
# ═══════════════════════════════════════════════════════════

print("=" * 70)
print("📊 CSV 파일 읽기 시작")
print("=" * 70)

df = pd.read_csv(CSV_FILE, encoding='utf-8-sig')

# 사원명을 키로 하는 딕셔너리 생성
employee_dict = {}
for _, row in df.iterrows():
    employee_dict[row['사원명']] = {
        'emailId': row['ID'],
        'deptName': row['부서'],
        'empNo': row['사원번호'] if pd.notna(row['사원번호']) else '',
        'positionName': row['직위'] if pd.notna(row['직위']) else '',
        'deptCode': row['부서코드'] if pd.notna(row['부서코드']) else ''
    }

print(f"✅ 총 {len(employee_dict)}명의 사원 정보 로드 완료\n")

# ═══════════════════════════════════════════════════════════
# 2. 업데이트 함수들 (기존과 동일)
# ═══════════════════════════════════════════════════════════

# activities용 업데이트 함수 (퇴사자는 이름만 남기고 모두 공란)
def update_activity_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: 정상 업데이트
            person_obj['emailId'] = employee_dict[name]['emailId']
            person_obj['deptName'] = employee_dict[name]['deptName']
            person_obj['positionName'] = employee_dict[name]['positionName']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: 이름만 남기고 모두 공란
            person_obj['emailId'] = ''
            person_obj['deptName'] = ''
            person_obj['positionName'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# referrers용 업데이트 함수 (퇴사자는 이름만 남기고 모두 공란)
def update_referrer_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            # 현직자: empNo와 deptCode 업데이트
            person_obj['empNo'] = employee_dict[name]['empNo']
            person_obj['deptCode'] = employee_dict[name]['deptCode']
        else:
            # 퇴사자: 이름만 남기고 모두 공란
            person_obj['empNo'] = ''
            person_obj['deptCode'] = ''
    return person_obj

# ═══════════════════════════════════════════════════════════
# 3. DB 연결 및 데이터 읽기
# ═══════════════════════════════════════════════════════════

print("=" * 70)
print("💾 DB 연결 및 데이터 읽기 (2015년 문서만)")
print("=" * 70)

try:
    # DB 연결
    conn = pymysql.connect(
        host=DB_CONFIG['host'],
        user=DB_CONFIG['user'],
        password=DB_CONFIG['password'],
        database=DB_CONFIG['database'],
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    print(f"✅ DB 연결 성공: {DB_CONFIG['host']}:{DB_CONFIG['database']}\n")
    
    cursor = conn.cursor()
    
    # documents 테이블에서 2015년 문서만 읽기
    select_sql = """
    SELECT id, source_id, drafter_name, drafter_position, drafter_dept, 
           drafter_email, drafter_dept_code,
           activities, referrers, end_year
    FROM documents
    WHERE end_year = 2015
    """
    
    cursor.execute(select_sql)
    rows = cursor.fetchall()
    
    print(f"✅ 2015년 문서 {len(rows)}건 로드 완료\n")
    
    # ═══════════════════════════════════════════════════════════
    # 4. 데이터 업데이트 처리
    # ═══════════════════════════════════════════════════════════
    
    print("=" * 70)
    print("🔄 사원 정보 업데이트 시작 (2015년 문서만)")
    print("=" * 70)
    
    update_sql = """
    UPDATE documents 
    SET drafter_position = %s,
        drafter_dept = %s,
        drafter_email = %s,
        drafter_dept_code = %s,
        activities = %s,
        referrers = %s
    WHERE id = %s AND end_year = 2015
    """
    
    updated_count = 0
    error_count = 0
    
    for idx, row in enumerate(rows, 1):
        try:
            doc_id = row['id']
            drafter_name = row['drafter_name']
            
            # drafter 정보 업데이트 (퇴사자 특별 처리!)
            if drafter_name and drafter_name in employee_dict:
                # 현직자: 정상 업데이트
                drafter_position = employee_dict[drafter_name]['positionName']
                drafter_dept = employee_dict[drafter_name]['deptName']
                drafter_email = employee_dict[drafter_name]['emailId']
                drafter_dept_code = employee_dict[drafter_name]['deptCode']
            else:
                # 퇴사자 또는 이름 없음: 특별 처리!
                drafter_position = ''
                drafter_dept = ''
                drafter_email = 'master'  # ← 퇴사자는 master!
                drafter_dept_code = ''
            
            # activities JSON 업데이트
            activities_text = row.get('activities', '[]')
            try:
                activities = json.loads(activities_text) if activities_text else []
            except:
                activities = []
            
            for activity in activities:
                update_activity_info(activity)
            
            updated_activities = json.dumps(activities, ensure_ascii=False)
            
            # referrers JSON 업데이트
            referrers_text = row.get('referrers', '[]')
            try:
                referrers = json.loads(referrers_text) if referrers_text else []
            except:
                referrers = []
            
            for referrer in referrers:
                update_referrer_info(referrer)
            
            updated_referrers = json.dumps(referrers, ensure_ascii=False)
            
            # UPDATE 실행
            cursor.execute(update_sql, (
                drafter_position,
                drafter_dept,
                drafter_email,
                drafter_dept_code,
                updated_activities,
                updated_referrers,
                doc_id
            ))
            
            updated_count += 1
            
            # 진행 상황 출력 (100건마다)
            if idx % 100 == 0:
                print(f"  진행 중... {idx}/{len(rows)} (성공: {updated_count}, 실패: {error_count})")
            
        except Exception as e:
            error_count += 1
            if error_count <= 5:
                print(f"  ❌ 오류 (ID: {row.get('id')}): {str(e)[:100]}")
    
    # 커밋
    print("\n💾 변경사항 커밋 중...")
    conn.commit()
    print("✅ 커밋 완료\n")
    
    # ═══════════════════════════════════════════════════════════
    # 5. 결과 요약
    # ═══════════════════════════════════════════════════════════
    
    print("=" * 70)
    print("✅ 업데이트 완료! (2015년 문서만)")
    print("=" * 70)
    print(f"총 문서: {len(rows)}건")
    print(f"성공: {updated_count}건")
    print(f"실패: {error_count}건")
    print("=" * 70)
    
    # 샘플 확인
    if rows:
        cursor.execute("""
        SELECT id, source_id, drafter_name, drafter_position, drafter_dept, 
               drafter_email, drafter_dept_code,
               activities, referrers, end_year
        FROM documents
        WHERE end_year = 2015
        LIMIT 1
        """)
        sample = cursor.fetchone()
        
        print("\n📋 업데이트 결과 샘플 (2015년 첫 번째 문서):")
        print(f"  ID: {sample['id']}")
        print(f"  Source ID: {sample['source_id']}")
        print(f"  연도: {sample['end_year']}")
        print(f"  기안자: {sample['drafter_name']} / {sample['drafter_position']} / {sample['drafter_dept']}")
        print(f"  기안자 이메일: {sample['drafter_email']}")
        print(f"  기안자 부서코드: {sample['drafter_dept_code']}")
        
        # activities 샘플
        try:
            activities = json.loads(sample['activities']) if sample['activities'] else []
            if activities:
                print(f"\n  Activities 첫 번째:")
                print(f"    {json.dumps(activities[0], ensure_ascii=False, indent=4)}")
        except:
            pass
        
        # referrers 샘플
        try:
            referrers = json.loads(sample['referrers']) if sample['referrers'] else []
            if referrers:
                print(f"\n  Referrers 첫 번째:")
                print(f"    {json.dumps(referrers[0], ensure_ascii=False, indent=4)}")
        except:
            pass
    
except pymysql.Error as e:
    print(f"\n❌ DB 오류 발생:")
    print(f"  Error Code: {e.args[0]}")
    print(f"  Error Message: {e.args[1]}")

except Exception as e:
    print(f"\n❌ 일반 오류 발생: {e}")
    import traceback
    traceback.print_exc()

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()
        print("\n✅ DB 연결 종료")

print("\n" + "=" * 70)
print("🎉 프로그램 종료")
print("=" * 70)

📊 CSV 파일 읽기 시작
✅ 총 156명의 사원 정보 로드 완료

💾 DB 연결 및 데이터 읽기 (2015년 문서만)
✅ DB 연결 성공: localhost:any_approval

✅ 2015년 문서 6587건 로드 완료

🔄 사원 정보 업데이트 시작 (2015년 문서만)
  진행 중... 100/6587 (성공: 100, 실패: 0)
  진행 중... 200/6587 (성공: 200, 실패: 0)
  진행 중... 300/6587 (성공: 300, 실패: 0)
  진행 중... 400/6587 (성공: 400, 실패: 0)
  진행 중... 500/6587 (성공: 500, 실패: 0)
  진행 중... 600/6587 (성공: 600, 실패: 0)
  진행 중... 700/6587 (성공: 700, 실패: 0)
  진행 중... 800/6587 (성공: 800, 실패: 0)
  진행 중... 900/6587 (성공: 900, 실패: 0)
  진행 중... 1000/6587 (성공: 1000, 실패: 0)
  진행 중... 1100/6587 (성공: 1100, 실패: 0)
  진행 중... 1200/6587 (성공: 1200, 실패: 0)
  진행 중... 1300/6587 (성공: 1300, 실패: 0)
  진행 중... 1400/6587 (성공: 1400, 실패: 0)
  진행 중... 1500/6587 (성공: 1500, 실패: 0)
  진행 중... 1600/6587 (성공: 1600, 실패: 0)
  진행 중... 1700/6587 (성공: 1700, 실패: 0)
  진행 중... 1800/6587 (성공: 1800, 실패: 0)
  진행 중... 1900/6587 (성공: 1900, 실패: 0)
  진행 중... 2000/6587 (성공: 2000, 실패: 0)
  진행 중... 2100/6587 (성공: 2100, 실패: 0)
  진행 중... 2200/6587 (성공: 2200, 실패: 0)
  진행 중... 2300/6587 (성공: 23